In [6]:
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd


In [7]:
wine=pd.read_csv("wine.scale",header=None,sep='\s+')
#print(wine)
for i in range(9):
    wine[i+1] = wine[i+1].str[2:]
ints=[10, 11, 12, 13]
for idx, val in enumerate(ints):
    wine[val] = wine[val].str[3:]

#for i in range(14):
    #print(wine[10])
#    wine[i] = wine[i].astype(float)

wine = wine.convert_objects(convert_numeric=True)
#wine

/home/disha/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [10]:
wine.to_csv("wine.csv", sep=',')

In [11]:
X = wine
X=X.drop(X.columns[[0]], axis=1)
y=wine[0]
X=X.fillna(X.median())
y

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
148    3
149    3
150    3
151    3
152    3
153    3
154    3
155    3
156    3
157    3
158    3
159    3
160    3
161    3
162    3
163    3
164    3
165    3
166    3
167    3
168    3
169    3
170    3
171    3
172    3
173    3
174    3
175    3
176    3
177    3
Name: 0, dtype: int64

In [87]:
###split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0) 

###build linear SVM
svc_rbf = svm.SVC(kernel='rbf', gamma=0.03,C=3)
#svc_multiclass=OneVsRestClassifier(LinearSVC(random_state=0))
#svc_multiclass.fit(X_train, y_train).predict(X_train)
#len(y_test)

In [88]:
###set max iterations T
T=25

#initialize sample weights
dm=np.repeat(1/len(X_train), len(X_train), axis=0)
#svc_linear.fit(X_train, y_train,sample_weight=dm)

#predict with initialized weights
predicted= svc_rbf.fit(X_train, y_train,sample_weight=dm).predict(X_test)

#print((predicted))
#print (dm)
#X
#np.where(predicted==2)[0].tolist()

In [89]:
###aplha parameter from adaboost algorith 
alpha=[]

###list to store predictions
pred=[]

###list to store test vector predictions
testpred=[]

###define number of classes
classes=3

###initialize the cost vector for class imbalance
#cost=np.random.uniform(0,1,3).tolist()
#cost=[]
#for i in range(classes):
#    cost.append(np.random.uniform(0,1,1).tolist())
    

###run adabosst multiclass with cost vector to each class
for t in range(T):
    #print(t)
    svc_rbf = svm.SVC(kernel='rbf', gamma=0.03,C=3)
    #fit a linear svm with weights dm
    svc_rbf.fit(X_train, y_train,sample_weight=dm)
    
    #predict train values
    predicted_train=(svc_rbf.predict(X_train))
    
    #predict test values
    predicted_test=(svc_rbf.predict(X_test))
    
    #store predicted test values
    testpred.append(predicted_test)
    
    #store predicted train values
    pred.append(predicted_train)
    
    #initially for each iteration error==0
    err=0
    
    #for each example in test predictions check misclassification
    for i in range(len(X_test)):
        if (testpred[t][i] != y_test.values[i]):
            y=y_train.values[i]
            err+=dm[i] #* cost[y-1]
    #calculate alpha wth err value and classes
    alpha.append(np.log((1-err)/err) + np.log(classes-1))
    
    #calculate Z value for normaliztion
    Z=[]
    #print(err)
    for i in range(len(X_train)):
        Z.append(dm[i] * np.exp((-alpha[t])*y_train.values[i] * predicted_train[i]))
    
    #update the weights
    dm=[Z[i] / sum(Z) for i in range(len(X_train))]

#initialize the adaboost ensemble otput for each class
ada_class=[]

#run for loop for number of classes
for k in range(classes):
    
    #initialize predictions for each test vector
    ada_class_pred=[]
    
    #for each test vector 
    for i in range(len(testpred[t])):
        
        #initialize the adaboost output as zero for each class
        ada_predicts=0
        #calculate the adaboost predicted value across t generations.this is a scalar
        for t in range(10):
            vec=testpred[t]
            al=alpha[t]
            if (vec[i]==k+1):
                ada_predicts+=(al *1)
            else:
                ada_predicts+=(al *0)              
        ada_class_pred.append(ada_predicts)
    #predictions of each test vector across classes
    ada_class.append(ada_class_pred)


In [90]:
#ada_class
#use vstack for better handling
adaclass_array=np.vstack(ada_class)
#initialize final pprediction
y_preds=[]
#for each test vector
for i in range(len(y_test)):
    #see for which class is adaboost prediction the maximum...and return that class
    y_preds.append(np.where(adaclass_array[:,i]==np.max(adaclass_array[:,i]))[0].tolist()[0]  + 1) #plus one because outputs as 1,2,3 needed
#y_preds


In [91]:
y_preds

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [92]:
len(y_preds)

54

In [93]:
y=y_test.tolist() 
acc=0
for i in range(len(y)):
    if(y[i] == y_preds[i]):
        acc+=1
accuracy=acc/100

accuracy

0.19